## this homework 1 for machine learning lecture

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.svm as svm
from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
import scipy.spatial.distance
from collections import Counter


In [7]:
## implementation of knn classsifer without sklearn whic takes k and metric params
class KNNClassifier:

    def __init__(self, k=5, metric='euclidean'):
        self.k = k
        self.metric = metric
        self.X_train = None
        self.y_train = None
        self.X_test = None
        self.y_test = None
        self.classes = None

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.classes = y_train
        return self

    def predict(self, X_test):
        self.X_test = X_test
        y_pred = []
        for i in range(self.X_test.shape[0]):
            # calculate distances
            distances = scipy.spatial.distance.cdist(self.X_train,
                                                     self.X_test[i, :].reshape(
                                                         1, -1),
                                                     metric=self.metric)
            # sort the distances
            sorted_indices = np.argsort(distances, axis=0)
            # get the indices of the k nearest neighbors
            k_nearest_indices = sorted_indices[:self.k]
            # get the classes of the k nearest neighbors
            # print(k_nearest_indices.flatten())
            k_nearest_classes = [
                self.y_train.get(i) for i in k_nearest_indices.flatten()
            ]

            # get the most common class
            most_common_class = Counter(k_nearest_classes).most_common(1)[0][0]
            y_pred.append(most_common_class)
        return y_pred

    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return np.sum(y_pred == y_test) / len(y_test)


In [8]:
bank_data = pd.read_csv('bank.csv', sep=';')
bank_data = bank_data.dropna()

Y = bank_data['y']
X = bank_data.drop(['y'], axis=1)
Y[1319]

'yes'

In [9]:
ordinalEncoder = OrdinalEncoder()
oneHotEncoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
labelEncoder = LabelEncoder()

column_transformer = make_column_transformer(
    (ordinalEncoder,
     ['job', 'education', 'housing', 'loan', 'contact', 'month', 'poutcome']),
    (oneHotEncoder, [
        'age',
        'campaign',
        'pdays',
        'previous',
        'day',
        'marital',
        'default',
    ]),
    remainder='passthrough')

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                    )
y_train[1310:1325]


3097     no
3716     no
109      no
3824     no
2924    yes
2622     no
4228     no
2308     no
459      no
4309     no
275      no
4384     no
3069     no
4137     no
1891     no
Name: y, dtype: object

In [10]:
pipeline = Pipeline([('column_transformer', column_transformer),
                     ('scaler', StandardScaler()),
                     ('classifier', KNNClassifier(k=4))])

pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
grid = GridSearchCV(pipeline,
                   param_grid={'classifier__k': [4]},
                    cv=cv)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)

print(grid.best_params_)
print(grid.best_score_)
print(grid.score(X_test, y_test))



{'classifier__k': 4}
0.46488479262672816
0.7447513812154696
